---
title: Tracking Changes
date: 2023-11-30 
authors:
  - name: Sébastien Boisgérault
    email: Sebastien.Boisgerault@minesparis.psl.eu
    url: https://github.com/boisgera
    affiliations:
      - institution: Mines Paris - PSL University
        department: Institut des Transformation Numériques (ITN)
github: boisgera
license: CC-BY-4.0
open_access: true
---

In order to understand how `.tldr` files are structured, we can add a new graphical objects, change some of their properties, etc. and each time we modify the document, analyze the corresponding evolution of the file.

In this notebook, we develop some tooling to help us track such changes.

## Text comparison

We define two similar versions of the "zen of Python":

In [7]:
zen_1 = """The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Errors should never pass silently.
In the face of ambiguity, refuse the temptation to guess.
There should be one obvious way to do it.
Although that way may not be obvious at first.
Now is better than never.
Although never is often better than right now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it is a good idea.
"""

In [8]:
zen_2 = """\
The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
"""

```{exercise}
 1. Transform `zen_1` and `zen_2` into list of lines.
 2. Use the [`difflib`](https://docs.python.org/3/library/difflib.html) module of the Python standard library to [`compare`](https://docs.python.org/3/library/difflib.html#difflib.Differ.compare) the two sequences.
 3. Make a text out of the output of compare and print it.
 4. Interpret the result and list the differences between both versions of the zen of Python.
```

In [9]:
#1
zen_1_l = zen_1.splitlines()
zen_2_l = zen_2.splitlines()

#print(zen_1_l)
#print(zen_2_l)

In [13]:
#2
import difflib
from difflib import context_diff, Differ, ndiff

In [14]:
#help(difflib)

In [15]:
#3
d = difflib.Differ()
diff = d.compare(zen_1_l,zen_2_l)

lst = []
for line in diff:
    print(line)
    lst.append(line)

  The Zen of Python, by Tim Peters
  
  Beautiful is better than ugly.
  Explicit is better than implicit.
  Simple is better than complex.
  Complex is better than complicated.
  Flat is better than nested.
  Sparse is better than dense.
  Readability counts.
  Special cases aren't special enough to break the rules.
+ Although practicality beats purity.
  Errors should never pass silently.
+ Unless explicitly silenced.
  In the face of ambiguity, refuse the temptation to guess.
- There should be one obvious way to do it.
+ There should be one-- and preferably only one --obvious way to do it.
- Although that way may not be obvious at first.
+ Although that way may not be obvious at first unless you're Dutch.
?                                              ++++++++++++++++++++

  Now is better than never.
- Although never is often better than right now.
+ Although never is often better than *right* now.
?                                     +     +

  If the implementation is hard to exp

In [16]:
#4
#'+' represent the lines where something has been added in zen_2_l compared to zen_1_l, and '-' when something has been erased.
line_nb = 0

for i in lst:
    if '+' in i or '^' in i or '-' in i:
        print(f'{line_nb} :',i)
        line_nb += 1
    else:
        line_nb += 1

10 : + Although practicality beats purity.
12 : + Unless explicitly silenced.
14 : - There should be one obvious way to do it.
15 : + There should be one-- and preferably only one --obvious way to do it.
16 : - Although that way may not be obvious at first.
17 : + Although that way may not be obvious at first unless you're Dutch.
18 : ?                                              ++++++++++++++++++++

20 : - Although never is often better than right now.
21 : + Although never is often better than *right* now.
22 : ?                                     +     +

24 : - If the implementation is easy to explain, it is a good idea.
25 : ?                                              ^^

26 : + If the implementation is easy to explain, it may be a good idea.
27 : ?                                              ^^^^^^

28 : + Namespaces are one honking great idea -- let's do more of those!


We can make our job easier if we use HTML instead of plain text to visualise the differences between the two texts.


```{exercise}
  1. Use the [HtmlDiff](https://docs.python.org/3/library/difflib.html#difflib.HtmlDiff) class of difflib to produce a `diff.html` file that represents this difference in a HTML document.
  2. Use the [webbrowser](https://docs.python.org/3/library/webbrowser.html) module of the standard library to open it!
  3. Define a `display_diff_text` function that takes two arguments `text_1` and `text_2` and automates steps 1. and 2.
```

In [17]:
#1
from difflib import HtmlDiff

h = HtmlDiff()
html_file = h.make_file(zen_1_l,zen_2_l)

with open('docs/diff.html', mode='w') as file:
    file.write(html_file)

In [18]:
#2
import webbrowser

webbrowser.open('docs/diff.html')

True

In [19]:
#3
from difflib import HtmlDiff
import webbrowser

def display_diff_text(text_1,text_2,file_path):
    h = HtmlDiff()
    html_file = h.make_file(text_1,text_2)
    
    with open(file_path, mode='w') as file:
        file.write(html_file)

    webbrowser.open(file_path)

## Comparison of JSON documents

````{exercise} Comparison of dictionnaries

 1. Create a `display_diff` function that takes two Python objects, converts them to strings then leverages `display_diff_text` to display the difference in a browser.

 2. Consider the 3 dictionaries defined by
    ```python
    d1 = {k:k+1 for k in range(100)}
    d2 = d1.copy(); d2[50] = 50
    d3 = {k:k+1 for k in range(99, -1, -1)}
    ```
    `d1` and `d2` have a slight difference and `d1` and `d3` are equal.
    Does your `display_diff` function make easy to spot where the difference is in the first case when it compares `d1` and `d2`?
    Does it make easy to see that `d1` and `d3` are equal?

  3. Investigate the [`pprint`](https://docs.python.org/3/library/pprint.html) module standard library ; use it to improve the behavior of `display_text_diff` in the two cases considered in the previous question.

````
 

In [40]:
#1
def display_diff(obj_1,obj_2,file_path):
    str1, str2 = str(obj_1), str(obj_2)

    display_diff_text(str1,str2,file_path)

#2
d1 = {k:k+1 for k in range(100)}
d2 = d1.copy(); d2[50] = 50
d3 = {k:k+1 for k in range(99, -1, -1)}

In [41]:
display_diff(d1,d3,'diffs_d1_d3.html')
#If we don't take an attentive look at the file 'diffs_d1_d3.html', it seems that d1 and d3 aren't equal.

In [49]:
#3
import pprint

pp = pprint.PrettyPrinter(sort_dicts=True)
d1s, d2s, d3s = pp.pformat(d1), pp.pformat(d2), pp.pformat(d3)

In [48]:
display_diff(d1s,d3s,'diffs_d1s_d3s.html')

```{exercise} tldraw documents comparator
Implement a function `tldraw_diff` that takes as argument two filenames that refer to tldraw documents and display their differences in the browser.
```

In [60]:
def tldraw_diff(file_1, file_2, file_path_4_display):
    from difflib import HtmlDiff
    import pprint
    import webbrowser
    import json

    f1_tldr = open(file_1, mode='r')
    f1_py = json.loads(f1_tldr.read())

    f2_tldr = open(file_2, mode='r')
    f2_py = json.loads(f2_tldr.read())

    f1pypp = pprint.PrettyPrinter(sort_dicts=True).pformat(f1_py)
    f2pypp = pprint.PrettyPrinter(sort_dicts=True).pformat(f2_py)

    
    with open(file_path_4_display, mode='w') as show_diff:
        html_file = HtmlDiff().make_file(f1pypp,f2pypp) #passe en html et montre les diff
        show_diff.write(html_file) #sauvegarde le fichier

    webbrowser.open(file_path_4_display)
    

In [61]:
tldraw_diff('hello-python.tldr','hello-tldraw.tldr','diff_tldr.html')